# Keras后端
## 什么是“后端”

Keras是一个模型级的库，提供了快速构建深度学习网络的模块。Keras并不处理如张量乘法、卷积等底层操作。这些操作依赖于某种特定的、优化良好的张量操作库。Keras依赖于处理张量的库就称为“后端引擎”。Keras提供了两种后端引擎Theano/Tensorflow，并将其函数统一封装，使得用户可以以同一个接口调用不同后端引擎的函数

* Theano是一个开源的符号主义张量操作框架，由蒙特利尔大学LISA/MILA实验室开发
* TensorFlow是一个符号主义的张量操作框架，由Google开发

在未来，我们有可能要添加更多的后端选项，如果你有兴趣开发后端，请与我联系~

## 切换后端

如果你至少运行过一次Keras，你将在下面的目录下找到Keras的配置文件：

  ~/.keras/keras.json

如果该目录下没有该文件，你可以手动创建一个

文件的默认配置如下：

{
"image_dim_ordering":"tf",
"epsilon":1e-07,
"floatx":"float32",
"backend":"tensorflow"
}

将backend字段的值改写为你需要使用的后端：theano或tensorflow，即可完成后端的切换

我们也可以通过定义环境变量KERAS_BACKEND来覆盖上面配置文件中定义的后端：

KERAS_BACKEND=tensorflow python -c "from keras import backend;"
Using TensorFlow backend.

## keras.json 细节

{
    "image_dim_ordering": "tf",
    "epsilon": 1e-07,
    "floatx": "float32",
    "backend": "tensorflow"
}

你可以更改以上~/.keras/keras.json中的配置
* image_dim_ordering：字符串，"tf"或"th"，该选项指定了Keras将要使用的维度顺序，可通过keras.backend.image_dim_ordering()来获取当前的维度顺序。对2D数据来说，tf假定维度顺序为(rows,cols,channels)而th假定维度顺序为(channels, rows, cols)。对3D数据而言，tf假定(conv_dim1, conv_dim2, conv_dim3, channels)，th则是(channels, conv_dim1, conv_dim2, conv_dim3)
* epsilon：浮点数，防止除0错误的小数字
* floatx：字符串，"float16", "float32", "float64"之一，为浮点数精度
* backend：字符串，所使用的后端，为"tensorflow"或"theano"


## 使用抽象的Keras后端来编写代码

如果你希望你编写的Keras模块能够同时在Theano和TensorFlow两个后端上使用，你可以通过Keras后端接口来编写代码，这里是一个简介：

In [1]:
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [2]:
#下面的代码实例化了一个输入占位符，等价于tf.placeholder() ，T.matrix()，T.tensor3()等

input = K.placeholder(shape=(2, 4, 5))
# also works:
input = K.placeholder(shape=(None, 4, 5))
# also works:
input = K.placeholder(ndim=3)

In [3]:
#下面的代码实例化了一个共享变量（shared），等价于tf.variable()或 theano.shared()

val = np.random.random((3, 4, 5))
var = K.variable(value=val)
print('val:',val)
print('var:',var)
print('var.eval():',K.eval(var))
# all-zeros variable:
var = K.zeros(shape=(3, 4, 5))
# all-ones:
var = K.ones(shape=(3, 4, 5))

scalar=K.variable(value=1)   #标量
print('type(scalar):',type(scalar))
print('scalar.shape:',scalar.shape)
print('scalar.eval():',K.eval(scalar))

vec=K.variable(value=[1,2,3])#向量
print('type(vec):',type(vec))
print('vec.shape:',vec.shape)
print('vec.eval():',K.eval(vec))

val: [[[0.79496233 0.54872479 0.08859713 0.35435715 0.03394271]
  [0.50680406 0.32313895 0.87309698 0.89853147 0.93612113]
  [0.31808792 0.24273923 0.27757798 0.93155299 0.4561912 ]
  [0.6233277  0.77349137 0.00133366 0.01985535 0.43858308]]

 [[0.19338747 0.49391168 0.6575116  0.28903274 0.06796342]
  [0.24399184 0.43370948 0.09434266 0.6000801  0.87372022]
  [0.48281087 0.14385422 0.5573031  0.49824132 0.56656246]
  [0.15673988 0.35300013 0.2813824  0.80561722 0.4395718 ]]

 [[0.58334764 0.22605754 0.30405116 0.11582888 0.45504755]
  [0.46195351 0.92311736 0.44215206 0.4578755  0.98691677]
  [0.1913837  0.15435109 0.8365811  0.52367146 0.85430744]
  [0.51205048 0.76778    0.75212    0.00719794 0.66528514]]]
var: <tf.Variable 'Variable:0' shape=(3, 4, 5) dtype=float32_ref>
var.eval(): [[[0.79496235 0.5487248  0.08859713 0.35435715 0.03394271]
  [0.50680405 0.32313895 0.873097   0.8985315  0.9361211 ]
  [0.31808794 0.24273923 0.27757797 0.931553   0.45619118]
  [0.62332773 0.7734914  0

In [4]:
a=K.variable(value=[[1]])
b=K.variable(value=[[1]])
c=K.transpose(a)
d=K.dot(a,b) #点积必须是二维数组
e=K.dot(a,c)
print('a.eval():',K.eval(a))
print('b.eval():',K.eval(b))
print('c.eval():',K.eval(c))
print('d.eval():',K.eval(d))


a.eval(): [[1.]]
b.eval(): [[1.]]
c.eval(): [[1.]]
d.eval(): [[1.]]


In [5]:
#大多数你需要的张量操作都可以通过统一的Keras后端接口完成，而不关心具体执行这些操作的是Theano还是TensorFlow
#b=K.variable(value=[[1,2],[1,2]])
#c=K.variable(value=[[2,3],[2,3]])
#d=K.variable(value=[[3,4],[3,4]])
#a = b + c * K.abs(d)
#c = K.dot(a, K.transpose(b))
#a = K.sum(b, axis=2)
#a = K.softmax(b)
#a = concatenate([b, c], axis=-1)
# etc...

# Kera后端函数

In [6]:
#epsilon()
#  以数值形式返回一个（一般来说很小的）数，用以防止除0错误
print('K.epsilon():',K.epsilon())


K.epsilon(): 1e-07


In [8]:
#set_epsilon(e)
#  设置在数值表达式中使用的fuzz factor，用于防止除0错误，该值应该是一个较小的浮点数，示例：
print('K.epsilon():',K.epsilon())

K.set_epsilon(1e-05)
print('K.epsilon():',K.epsilon())
K.set_epsilon(1e-07)

K.epsilon(): 1e-05
K.epsilon(): 1e-05


In [9]:
#floatx()
#  返回默认的浮点数数据类型，为字符串，如 'float16', 'float32', 'float64'
print('K.floatx():',K.floatx())

K.floatx(): float32


In [10]:
#set_floatx(floatx)
#  设置默认的浮点数数据类型，为字符串，如 'float16', 'float32', 'float64',示例：
print('K.floatx():',K.floatx())
K.set_floatx('float16')
print('K.floatx():',K.floatx())

K.set_floatx('float32')

K.floatx(): float32
K.floatx(): float16


In [12]:
#cast_to_floatx(x)
#  将numpy array转换为默认的Keras floatx类型，x为numpy array，返回值也为numpy array但其数据类型变为floatx。示例：
print('K.floatx():',K.floatx())
arr = np.array([1.0, 2.0], dtype='float64')
print('arr.dtype:',arr.dtype)
new_arr = K.cast_to_floatx(arr)
print('new_arr.dtype:',new_arr.dtype)


K.floatx(): float32
arr.dtype: float64
new_arr.dtype: float32


In [13]:
#image_dim_ordering()
#  返回默认的图像的维度顺序（‘tf’或‘th’）
print('K.image_dim_ordering():',K.image_dim_ordering())

K.image_dim_ordering(): tf


In [14]:
#set_image_dim_ordering(dim_ordering)
#  设置图像的维度顺序（‘tf’或‘th’）,示例：
print('K.image_dim_ordering():',K.image_dim_ordering())
K.set_image_dim_ordering('th')
print('K.image_dim_ordering():',K.image_dim_ordering())
K.set_image_dim_ordering('tf')

K.image_dim_ordering(): tf
K.image_dim_ordering(): th


In [23]:
#get_uid(prefix='')
#  依据给定的前缀提供一个唯一的UID，参数为表示前缀的字符串，返回值为整数，示例：
print(K.get_uid('dense'))
print(K.get_uid('dense'))


12
13


In [42]:
#结果存疑？？？

#is_keras_tensor(x)
#  判断x是否是一个Keras tensor，返回一个布尔值，示例
np_var = np.array([1, 2])
#K.is_keras_tensor(np_var)
keras_var = K.variable(np_var)
print(K.is_keras_tensor(keras_var))  # A variable is not a Tensor.
keras_placeholder = K.placeholder(shape=(2, 4, 5))
print(K.is_keras_tensor(keras_placeholder))  # A placeholder is a Tensor.


False
False


In [44]:
#clear_session()
#  结束当前的TF计算图，并新建一个。有效的避免模型/层的混乱
K.clear_session()
a=K.variable(value=1)
K.eval(a)

1.0

In [ ]:
#manual_variable_initialization(value)
#  指出变量应该以其默认值被初始化还是由用户手动初始化，参数value为布尔值，默认False代表变量由其默认值初始化

In [52]:
#learning_phase()
#  返回训练模式/测试模式的flag，该flag是一个用以传入Keras模型的标记，以决定当前模型执行于训练模式下还是测试模式下
K.learning_phase()

0

In [53]:
#set_learning_phase(flag)
#  设置训练模式/测试模式0或1
K.set_learning_phase(1)
print(K.learning_phase())
K.set_learning_phase(0)
print(K.learning_phase())

1
0


In [54]:
#is_sparse(tensor)
#  判断一个tensor是不是一个稀疏的tensor(稀不稀疏由tensor的类型决定，而不是tensor实际上有多稀疏)，返回值是一个布尔值，示例：
a = K.placeholder((2, 2), sparse=False)
print(K.is_sparse(a))
b = K.placeholder((2, 2), sparse=True)
print(K.is_sparse(b))

False
True


In [55]:
#to_dense(tensor)
#  将一个稀疏tensor转换一个不稀疏的tensor并返回之，示例：
b = K.placeholder((2, 2), sparse=True)
print(K.is_sparse(b))
c = K.to_dense(b)
print(K.is_sparse(c))

True
False


In [60]:
#variable(value, dtype='float32', name=None)
#实例化一个张量，返回之
#参数：
#
#    value：用来初始化张量的值
#    dtype：张量数据类型
#    name：张量的名字（可选）
#
#示例：
val = np.array([[1, 2], [3, 4]])
kvar = K.variable(value=val, dtype='float64', name='example_var')
print('kvar',kvar)
print('kvar.eval():\n',K.eval(kvar))

#标量
scalar=K.variable(value=1,dtype='float32',name='scalar_var')
print('scalar:',scalar)
print('scalar.eval():\n',K.eval(scalar))

#向量
vec=K.variable(value=[1,2,3],dtype='float32',name='vector_var')
print('vec:',vec)
print('vec.eval():\n',K.eval(vec))

kvar <tf.Variable 'example_var_4:0' shape=(2, 2) dtype=float64_ref>
kvar.eval():
 [[1. 2.]
 [3. 4.]]
scalar: <tf.Variable 'scalar_var_1:0' shape=() dtype=float32_ref>
scalar.eval():
 1.0
vec: <tf.Variable 'vector_var:0' shape=(3,) dtype=float32_ref>
vec.eval():
 [1. 2. 3.]


In [61]:
#placeholder(shape=None, ndim=None, dtype='float32', name=None)
#实例化一个占位符，返回之
#参数：
#    shape：占位符的shape（整数tuple，可能包含None）
#    ndim: 占位符张量的阶数，要初始化一个占位符，至少指定shape和ndim之一，如果都指定则使用shape
#    dtype: 占位符数据类型
#    name: 占位符名称（可选）
#示例：
input_ph = K.placeholder(shape=(2, 4, 5))
print('input_ph:',input_ph)

input_ph: Tensor("Placeholder_5:0", shape=(2, 4, 5), dtype=float32)


In [64]:
#shape(x)
#  返回一个张量的符号shape，符号shape的意思是返回值本身也是一个tensor，示例：
tf_session = K.get_session()
val = np.array([[1, 2], [3, 4]])
kvar = K.variable(value=val)
print('K.shape(kvar):',K.shape(kvar))
input = K.placeholder(shape=(2, 4, 5))
print('K.shape(input):',K.shape(input))
print('K.shape(kvar).eval(session=tf_session):',K.shape(kvar).eval(session=tf_session))
print('K.shape(input).eval(session=tf_session):',K.shape(input).eval(session=tf_session))

K.shape(kvar): Tensor("Shape:0", shape=(2,), dtype=int32)
K.shape(input): Tensor("Shape_1:0", shape=(3,), dtype=int32)
K.shape(kvar).eval(session=tf_session): [2 2]
K.shape(input).eval(session=tf_session): [2 4 5]


In [65]:
#int_shape(x)
#  以整数Tuple或None的形式返回张量shape，示例：
input = K.placeholder(shape=(2, 4, 5))
print('K.int_shape(input):',K.int_shape(input))
val = np.array([[1, 2], [3, 4]])
kvar = K.variable(value=val)
print('K.int_shape(kvar):',K.int_shape(kvar))

K.int_shape(input): (2, 4, 5)
K.int_shape(kvar): (2, 2)


In [66]:
#ndim(x)
#  返回张量的阶数，为整数，示例：
input = K.placeholder(shape=(2, 4, 5))
val = np.array([[1, 2], [3, 4]])
kvar = K.variable(value=val)
print('K.ndim(input):',K.ndim(input))
print('K.ndim(kvar):',K.ndim(kvar))

K.ndim(input): 3
K.ndim(kvar): 2


In [67]:
#dtype(x)
#  返回张量的数据类型，为字符串，示例：
print('K.dtype(K.placeholder(shape=(2,4,5))):',K.dtype(K.placeholder(shape=(2,4,5))))
print("K.dtype(K.placeholder(shape=(2,4,5), dtype='float32')):",K.dtype(K.placeholder(shape=(2,4,5), dtype='float32')))
print("K.dtype(K.placeholder(shape=(2,4,5), dtype='float64')):",K.dtype(K.placeholder(shape=(2,4,5), dtype='float64')))
kvar = K.variable(np.array([[1, 2], [3, 4]]))
print('K.dtype(kvar):',K.dtype(kvar))
kvar = K.variable(np.array([[1, 2], [3, 4]]), dtype='float32')
print('K.dtype(kvar):',K.dtype(kvar))

K.dtype(K.placeholder(shape=(2,4,5))): float32
K.dtype(K.placeholder(shape=(2,4,5), dtype='float32')): float32
K.dtype(K.placeholder(shape=(2,4,5), dtype='float64')): float64
K.dtype(kvar): float32
K.dtype(kvar): float32


In [69]:
#eval(x)
#  求得张量的值，返回一个Numpy array，示例：
kvar = K.variable(np.array([[1, 2], [3, 4]]), dtype='float32')
print('K.eval(kvar):\n',K.eval(kvar))

K.eval(kvar):
 [[1. 2.]
 [3. 4.]]


In [71]:
#zeros(shape, dtype='float32', name=None)
#  生成一个全0张量，示例：
kvar = K.zeros((3,4))
print('K.eval(kvar):\n',K.eval(kvar))

K.eval(kvar):
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [73]:
#ones(shape, dtype='float32', name=None)
#  生成一个全1张量，示例
kvar = K.ones((3,4))
print('K.eval(kvar):\n',K.eval(kvar))

K.eval(kvar):
 [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


In [74]:
#eye(size, dtype='float32', name=None)
#  生成一个单位矩阵，示例：
kvar = K.eye(3)
print('K.eval(kvar):\n',K.eval(kvar))

K.eval(kvar):
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [75]:
#zeros_like(x, name=None)
#  生成与另一个张量x的shape相同的全0张量，示例：
kvar = K.variable(np.random.random((2,3)))
kvar_zeros = K.zeros_like(kvar)
print('K.eval(kvar_zeros):\n',K.eval(kvar_zeros))

K.eval(kvar_zeros):
 [[0. 0. 0.]
 [0. 0. 0.]]


In [76]:
#ones_like(x, name=None)
#  生成与另一个张量shape相同的全1张量，示例：
kvar = K.variable(np.random.random((2,3)))
kvar_ones = K.ones_like(kvar)
print('K.eval(kvar_ones):\n',K.eval(kvar_ones))

K.eval(kvar_ones):
 [[1. 1. 1.]
 [1. 1. 1.]]


In [78]:
#random_uniform_variable(shape, low, high, dtype=None, name=None, seed=None)
#初始化一个Keras变量，其数值为从一个均匀分布中采样的样本，返回之。
#参数：
#    shape：张量shape
#    low：浮点数，均匀分布之下界
#    high：浮点数，均匀分布之上界
#    dtype：数据类型
#    name：张量名
#    seed：随机数种子
#示例：
kvar = K.random_uniform_variable((2,3), 0, 1)
print('kvar:\n',kvar)
print('K.eval(kvar):\n',K.eval(kvar))

kvar:
 <tf.Variable 'Variable_16:0' shape=(2, 3) dtype=float32_ref>
K.eval(kvar):
 [[0.02021575 0.28859675 0.6882783 ]
 [0.6855142  0.4137349  0.17341638]]


In [79]:
#count_params(x)
#  返回张量中标量的个数，示例：
kvar = K.zeros((2,3))
print('K.count_params(kvar):',K.count_params(kvar))
print('K.eval(kvar):\n',K.eval(kvar))

K.count_params(kvar): 6
K.eval(kvar):
 [[0. 0. 0.]
 [0. 0. 0.]]


In [82]:
#cast(x, dtype)
#  改变张量的数据类型，dtype只能是float16, float32或float64之一，示例：
input = K.placeholder((2, 3), dtype='float32')
print('input:',input)
new1_input=K.cast(input, dtype='float16')
print('new1_input:',new1_input)
new2_input = K.cast(input, dtype=np.float16)
print('new2_input:',new2_input)

input: Tensor("Placeholder_14:0", shape=(2, 3), dtype=float32)
new1_input: Tensor("Cast_3:0", shape=(2, 3), dtype=float16)
new2_input: Tensor("Cast_4:0", shape=(2, 3), dtype=float16)


In [86]:
#dot(x, y)
#  求两个张量的乘积。当试图计算两个N阶张量的乘积时，与Theano行为相同，如(2, 3).(4, 3, 5) = (2, 4, 5))，示例：
x = K.placeholder(shape=(2, 3))
y = K.placeholder(shape=(3, 4))
xy = K.dot(x, y)
print('xy:',xy)

x = K.placeholder(shape=(32, 28, 3))
y = K.placeholder(shape=(3, 4))
xy = K.dot(x, y)
print('xy:',xy)

xy: Tensor("MatMul_8:0", shape=(2, 4), dtype=float32)
xy: Tensor("Reshape_11:0", shape=(32, 28, 4), dtype=float32)


In [106]:
#batch_dot(x, y, axes=None)
#  按批进行张量乘法，该函数用于计算x和y的点积，其中x和y都是成batch出现的数据。即它的数据shape形如(batch_size,:)。
#batch_dot将产生比输入张量维度低的张量，如果张量的维度被减至1，则通过expand_dims保证其维度至少为2 
#例如，假设x = [[1, 2],[3,4]] ， y = [[5, 6],[7, 8]]，则batch_dot(x, y, axes=1) = [[17, 53]]，
#即x.dot(y.T)的主对角元素，此过程中我们没有计算过反对角元素的值
#参数：
#    x,y：阶数大于等于2的张量，在tensorflow下，只支持大于等于3阶的张量
#    axes：目标结果的维度，为整数或整数列表，axes[0]和axes[1]应相同
#示例： 假设x=[[1,2],[3,4]]，y=[[5,6],[7,8]]，则batch_dot(x, y, axes=1)为[[17, 53]]，恰好为x.dot(y.T)的主对角元，
#整个过程没有计算反对角元的元素。
#我们做一下shape的推导，假设x是一个shape为(100,20)的tensor，y是一个shape为(100,30,20)的tensor，假设axes=(1,2)，
#则输出tensor的shape通过循环x.shape和y.shape确定：
#    x.shape[0]：值为100，加入到输入shape里
#    x.shape[1]：20，不加入输出shape里，因为该维度的值会被求和(dot_axes[0]=1)
#    y.shape[0]：值为100，不加入到输出shape里，y的第一维总是被忽略
#    y.shape[1]：30，加入到输出shape里#
#    y.shape[2]：20，不加到output shape里，y的第二个维度会被求和(dot_axes[1]=2)
#    结果为(100, 30)

x_batch = K.ones(shape=(32, 20, 1))
y_batch = K.ones(shape=(32, 30,20))
xy_batch_dot = K.batch_dot(x_batch, y_batch, axes=[1, 2])
print('K.int_shape(xy_batch_dot):',K.int_shape(xy_batch_dot))

x_batch = K.ones(shape=(2,2, 1))
y_batch = K.ones(shape=(2,3, 2))
xy_batch_dot = K.batch_dot(x_batch, y_batch, axes=[1,2])
print('K.int_shape(xy_batch_dot):',K.int_shape(xy_batch_dot))
print(K.eval(xy_batch_dot))



K.int_shape(xy_batch_dot): (32, 1, 30)
K.int_shape(xy_batch_dot): (2, 1, 3)
[[[2. 2. 2.]]

 [[2. 2. 2.]]]


In [107]:
#transpose(x)
#  张量转置，返回转置后的tensor，示例：
var = K.variable([[1, 2, 3], [4, 5, 6]])
print('K.eval(var):\n',K.eval(var))
var_transposed = K.transpose(var)
print('K.eval(var_transposed):\n',K.eval(var_transposed))

input = K.placeholder((2, 3))
print('input:',input)
input_transposed = K.transpose(input)
print('input_transposed:',input_transposed)

K.eval(var):
 [[1. 2. 3.]
 [4. 5. 6.]]
K.eval(var_transposed):
 [[1. 4.]
 [2. 5.]
 [3. 6.]]
input: Tensor("Placeholder_35:0", shape=(2, 3), dtype=float32)
input_transposed: Tensor("transpose_5:0", shape=(3, 2), dtype=float32)


In [116]:
#gather(reference, indices)
#  在给定的2D张量中检索给定下标的向量
#参数：
#    reference：2D张量
#    indices：整数张量，其元素为要查询的下标
#返回值：一个与reference数据类型相同的3D张量
np_val=np.random.random((3,4,5))
var=K.variable(value=np_val)
indices=K.variable(value=[0,1],dtype='int32')
gather_var=K.gather(var,indices)
print(var)
print(gather_var)

np_val=np.random.random((4,5))
var=K.variable(value=np_val)
indices=K.variable(value=[0,1],dtype='int32')
gather_var=K.gather(var,indices)
print(var)
print(gather_var)

<tf.Variable 'Variable_98:0' shape=(3, 4, 5) dtype=float32_ref>
Tensor("embedding_lookup_4:0", shape=(2, 4, 5), dtype=float32)
<tf.Variable 'Variable_100:0' shape=(4, 5) dtype=float32_ref>
Tensor("embedding_lookup_5:0", shape=(2, 5), dtype=float32)


In [123]:
#max(x, axis=None, keepdims=False)
#  求张量中的最大值
np_val=np.random.random((2,3,4))
print('np_val:\n',np_val)
x=K.variable(value=np_val)
max_x=K.max(x)
print('max_x:\n',K.eval(max_x))
max_x=K.max(x,axis=0)
print('max_x:\n',K.eval(max_x))
max_x=K.max(x,axis=1)
print('max_x:\n',K.eval(max_x))
max_x=K.max(x,axis=2)
print('max_x:\n',K.eval(max_x))

np_val:
 [[[1.31411829e-01 5.63953442e-01 3.17394308e-01 9.22746806e-01]
  [8.45743250e-03 2.55561525e-04 1.87091242e-01 6.36773350e-01]
  [3.38401413e-01 6.61877545e-01 1.88218115e-01 8.26177343e-01]]

 [[4.94803903e-01 2.35665208e-01 3.89225123e-01 9.47078016e-01]
  [1.94513753e-01 7.52559173e-01 3.69761544e-01 7.13435253e-01]
  [9.52236423e-01 4.58220393e-01 7.51677989e-01 1.17848928e-01]]]
max_x:
 0.9522364
max_x:
 [[0.4948039  0.56395346 0.38922513 0.947078  ]
 [0.19451375 0.7525592  0.36976156 0.71343523]
 [0.9522364  0.6618776  0.751678   0.82617736]]
max_x:
 [[0.3384014  0.6618776  0.31739432 0.9227468 ]
 [0.9522364  0.7525592  0.751678   0.947078  ]]
max_x:
 [[0.9227468  0.63677335 0.82617736]
 [0.947078   0.7525592  0.9522364 ]]


In [124]:
#min(x, axis=None, keepdims=False)
#  求张量中的最小值
np_val=np.random.random((2,3,4))
print('np_val:\n',np_val)
x=K.variable(value=np_val)
min_x=K.min(x)
print('min_x:\n',K.eval(min_x))
min_x=K.min(x,axis=0)
print('min_x:\n',K.eval(min_x))
min_x=K.min(x,axis=1)
print('min_x:\n',K.eval(min_x))
min_x=K.min(x,axis=2)
print('min_x:\n',K.eval(min_x))

np_val:
 [[[0.94847442 0.19557186 0.1675426  0.90220799]
  [0.67798362 0.45490861 0.10340898 0.45392399]
  [0.91264745 0.42802316 0.6381998  0.08298229]]

 [[0.89808619 0.87338439 0.63070408 0.07428057]
  [0.45907634 0.21111644 0.59305023 0.72040462]
  [0.41785239 0.93421643 0.42992735 0.16800685]]]
min_x:
 0.074280575
min_x:
 [[0.8980862  0.19557185 0.1675426  0.07428057]
 [0.45907634 0.21111645 0.10340898 0.453924  ]
 [0.4178524  0.42802316 0.42992735 0.08298229]]
min_x:
 [[0.67798364 0.19557185 0.10340898 0.08298229]
 [0.4178524  0.21111645 0.42992735 0.07428057]]
min_x:
 [[0.1675426  0.10340898 0.08298229]
 [0.07428057 0.21111645 0.16800685]]


In [125]:
#sum(x, axis=None, keepdims=False)
#  在给定轴上计算张量中元素之和
np_val=np.random.random((2,3,4))
print('np_val:\n',np_val)
x=K.variable(value=np_val)
sum_x=K.sum(x)
print('sum_x:\n',K.eval(sum_x))
sum_x=K.sum(x,axis=0)
print('sum_x:\n',K.eval(sum_x))
sum_x=K.sum(x,axis=1)
print('sum_x:\n',K.eval(sum_x))
sum_x=K.sum(x,axis=2)
print('sum_x:\n',K.eval(sum_x))

np_val:
 [[[0.17712808 0.96672699 0.18601349 0.96360343]
  [0.90528473 0.59740785 0.05772159 0.41617274]
  [0.6886427  0.41530343 0.26504028 0.12073365]]

 [[0.30981784 0.1741021  0.17489191 0.71479188]
  [0.30590318 0.58069403 0.88076418 0.0220839 ]
  [0.99017032 0.97092972 0.78773541 0.88923502]]]
sum_x:
 12.560899
sum_x:
 [[0.48694593 1.1408291  0.3609054  1.6783953 ]
 [1.2111878  1.1781019  0.93848574 0.43825665]
 [1.678813   1.3862332  1.0527756  1.0099686 ]]
sum_x:
 [[1.7710555  1.9794383  0.50877535 1.5005099 ]
 [1.6058912  1.7257259  1.8433914  1.6261108 ]]
sum_x:
 [[2.293472  1.9765869 1.48972  ]
 [1.3736038 1.7894453 3.6380706]]


In [126]:
#prod(x, axis=None, keepdims=False)
#  在给定轴上计算张量中元素之积
np_val=np.random.random((2,3,4))
print('np_val:\n',np_val)
x=K.variable(value=np_val)
prod_x=K.prod(x)
print('prod_x:\n',K.eval(prod_x))
prod_x=K.prod(x,axis=0)
print('prod_x:\n',K.eval(prod_x))
prod_x=K.prod(x,axis=1)
print('prod_x:\n',K.eval(prod_x))
prod_x=K.prod(x,axis=2)
print('prod_x:\n',K.eval(prod_x))

np_val:
 [[[0.34574024 0.66155685 0.38360644 0.26711819]
  [0.67762723 0.20639704 0.4490393  0.89007383]
  [0.71654525 0.29003523 0.72316524 0.57168247]]

 [[0.09039276 0.82544141 0.68799535 0.25255285]
  [0.44146853 0.47313552 0.48685166 0.19368599]
  [0.05247178 0.80529524 0.8618623  0.81355364]]]
prod_x:
 8.5161e-10
prod_x:
 [[0.03125241 0.5460764  0.26391944 0.06746146]
 [0.2991511  0.09765378 0.21861553 0.17239483]
 [0.0375984  0.233564   0.62326884 0.46509433]]
prod_x:
 [[0.16787435 0.03960239 0.12456837 0.13592032]
 [0.00209392 0.31450456 0.28868225 0.03979575]]
prod_x:
 [[0.02343724 0.05589909 0.08591852]
 [0.01296456 0.0196961  0.02962818]]


In [127]:
#var(x, axis=None, keepdims=False)
#  在给定轴上计算张量方差
np_val=np.random.random((2,3,4))
print('np_val:\n',np_val)
x=K.variable(value=np_val)
var_x=K.var(x)
print('var_x:\n',K.eval(var_x))
var_x=K.var(x,axis=0)
print('var_x:\n',K.eval(var_x))
var_x=K.var(x,axis=1)
print('var_x:\n',K.eval(var_x))
var_x=K.var(x,axis=2)
print('var_x:\n',K.eval(var_x))

np_val:
 [[[0.35284558 0.88251151 0.31220217 0.10453733]
  [0.24418881 0.94908085 0.1567528  0.19312555]
  [0.87341624 0.20491909 0.0999985  0.56485595]]

 [[0.62442646 0.02635919 0.71567655 0.95794761]
  [0.23131146 0.78751582 0.31638346 0.54967745]
  [0.23266986 0.06654846 0.86820463 0.32407783]]]
var_x:
 0.09438578
var_x:
 [[1.8439047e-02 1.8324919e-01 4.0697888e-02 1.8207727e-01]
 [4.1456602e-05 6.5258127e-03 6.3704867e-03 3.1782310e-02]
 [1.0263898e-01 4.7866078e-03 1.4753518e-01 1.4493522e-02]]
var_x:
 [[0.07541414 0.11303774 0.00804622 0.0397694 ]
 [0.03422384 0.12230762 0.05413405 0.06881896]]
var_x:
 [[0.08234225 0.10673127 0.09355801]
 [0.11743488 0.04693227 0.09030651]]


In [128]:
#std(x, axis=None, keepdims=False)
#  在给定轴上求张量元素之标准差
np_val=np.random.random((2,3,4))
print('np_val:\n',np_val)
x=K.variable(value=np_val)
std_x=K.std(x)
print('std_x:\n',K.eval(std_x))
std_x=K.std(x,axis=0)
print('std_x:\n',K.eval(std_x))
std_x=K.std(x,axis=1)
print('std_x:\n',K.eval(std_x))
std_x=K.std(x,axis=2)
print('std_x:\n',K.eval(std_x))

np_val:
 [[[0.05977404 0.20152378 0.52515561 0.481173  ]
  [0.77147087 0.20253868 0.30459003 0.17935439]
  [0.96887069 0.92178327 0.70491862 0.62311386]]

 [[0.81551875 0.63817568 0.50782808 0.55483277]
  [0.18868109 0.50049538 0.97294642 0.52714189]
  [0.27565026 0.93928177 0.74023229 0.64846093]]]
std_x:
 0.2694572
std_x:
 [[0.37787238 0.21832594 0.00866377 0.03682987]
 [0.2913949  0.14897834 0.33417818 0.17389373]
 [0.34661022 0.00874925 0.01765683 0.01267353]]
std_x:
 [[0.3904322  0.33929458 0.16371618 0.18504177]
 [0.27727783 0.18322852 0.1898838  0.05190947]]
std_x:
 [[0.19350804 0.2396481  0.14455998]
 [0.11732095 0.27945456 0.24081248]]


In [129]:
#mean(x, axis=None, keepdims=False)
#  在给定轴上求张量元素之均值
np_val=np.random.random((2,3,4))
print('np_val:\n',np_val)
x=K.variable(value=np_val)
mean_x=K.mean(x)
print('mean_x:\n',K.eval(mean_x))
mean_x=K.mean(x,axis=0)
print('mean_x:\n',K.eval(mean_x))
mean_x=K.mean(x,axis=1)
print('mean_x:\n',K.eval(mean_x))
mean_x=K.mean(x,axis=2)
print('mean_x:\n',K.eval(mean_x))

np_val:
 [[[0.93956801 0.37005287 0.23761147 0.97642838]
  [0.96089167 0.95410779 0.32678373 0.01124815]
  [0.43376105 0.34766343 0.71113554 0.79964745]]

 [[0.22695161 0.75725425 0.22871023 0.80702581]
  [0.43823239 0.56308911 0.11457299 0.89704138]
  [0.28099254 0.99503055 0.89045146 0.86791718]]]
mean_x:
 0.5890071
mean_x:
 [[0.5832598  0.5636536  0.23316085 0.8917271 ]
 [0.699562   0.75859845 0.22067836 0.45414478]
 [0.3573768  0.67134696 0.8007935  0.8337823 ]]
mean_x:
 [[0.7780736  0.55727464 0.42517695 0.59577465]
 [0.31539217 0.7717913  0.4112449  0.8573281 ]]
mean_x:
 [[0.63091516 0.5632578  0.5730519 ]
 [0.50498545 0.50323397 0.75859797]]


In [136]:
#any(x, axis=None, keepdims=False)
#  按位或，返回数据类型为uint8的张量（元素为0或1）
np_val=np.random.randint(0,3,size=(2,3,4))
print('np_val:\n',np_val)
x=K.variable(value=np_val)
any_x=K.any(x)
print('any_x:\n',K.eval(any_x))
any_x=K.any(x,axis=0)
print('any_x:\n',K.eval(any_x))
any_x=K.any(x,axis=1)
print('any_x:\n',K.eval(any_x))
any_x=K.any(x,axis=2)
print('any_x:\n',K.eval(any_x))

np_val:
 [[[1 1 0 1]
  [1 1 1 1]
  [1 0 1 0]]

 [[1 1 0 1]
  [1 2 2 2]
  [1 0 2 0]]]
any_x:
 True
any_x:
 [[ True  True False  True]
 [ True  True  True  True]
 [ True False  True False]]
any_x:
 [[ True  True  True  True]
 [ True  True  True  True]]
any_x:
 [[ True  True  True]
 [ True  True  True]]


In [137]:
#all(x, axis=None, keepdims=False)
#  按位与，返回类型为uint8de tensor
np_val=np.random.randint(0,3,size=(2,3,4))
print('np_val:\n',np_val)
x=K.variable(value=np_val)
all_x=K.all(x)
print('all_x:\n',K.eval(all_x))
all_x=K.all(x,axis=0)
print('all_x:\n',K.eval(all_x))
all_x=K.all(x,axis=1)
print('all_x:\n',K.eval(all_x))
all_x=K.all(x,axis=2)
print('all_x:\n',K.eval(all_x))

np_val:
 [[[0 2 1 2]
  [2 2 1 2]
  [0 1 0 2]]

 [[2 0 0 1]
  [1 0 0 2]
  [0 0 1 0]]]
all_x:
 False
all_x:
 [[False False False  True]
 [ True False False  True]
 [False False False False]]
all_x:
 [[False  True False  True]
 [False False False False]]
all_x:
 [[False  True False]
 [False False False]]


In [138]:
#argmax(x, axis=-1)
#  在给定轴上求张量之最大元素下标
np_val=np.random.randint(0,3,size=(2,3,4))
print('np_val:\n',np_val)
x=K.variable(value=np_val)
argmax_x=K.argmax(x)
print('argmax_x:\n',K.eval(argmax_x))
argmax_x=K.argmax(x,axis=0)
print('argmax_x:\n',K.eval(argmax_x))
argmax_x=K.argmax(x,axis=1)
print('argmax_x:\n',K.eval(argmax_x))
argmax_x=K.argmax(x,axis=2)
print('argmax_x:\n',K.eval(argmax_x))

np_val:
 [[[2 1 0 2]
  [1 2 2 0]
  [0 2 0 2]]

 [[0 0 0 2]
  [0 2 1 0]
  [0 2 0 0]]]
argmax_x:
 [[0 1 1]
 [3 1 1]]
argmax_x:
 [[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
argmax_x:
 [[0 1 1 0]
 [0 1 1 0]]
argmax_x:
 [[0 1 1]
 [3 1 1]]


In [144]:
feats_val=np.random.random((2,3,4,5))
feats=K.variable(value=feats_val)
grid_shape = K.shape(feats)[1:3] # height, width; FeatureMap大小
grid_y = K.tile(K.reshape(K.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1]),[1, grid_shape[1], 1, 1])
grid_x = K.tile(K.reshape(K.arange(0, stop=grid_shape[1]), [1, -1, 1, 1]),[grid_shape[0], 1, 1, 1])
grid = K.concatenate([grid_x, grid_y])
grid = K.cast(grid, K.dtype(feats))
#print('feats_val:\n',feats_val)
print('feats:',feats)
print('grid_y:',grid_y)
print('grid_y.eval():\n',K.eval(grid_y))
print('grid_x:',grid_x)
print('grid_x.eval():\n',K.eval(grid_x))
print('grid:',grid)
print('grid.eval():',K.eval(grid))

feats: <tf.Variable 'Variable_124:0' shape=(2, 3, 4, 5) dtype=float32_ref>
grid_y: Tensor("Tile_8:0", shape=(3, 4, 1, 1), dtype=int32)
grid_y.eval():
 [[[[0]]

  [[0]]

  [[0]]

  [[0]]]


 [[[1]]

  [[1]]

  [[1]]

  [[1]]]


 [[[2]]

  [[2]]

  [[2]]

  [[2]]]]
grid_x: Tensor("Tile_9:0", shape=(3, 4, 1, 1), dtype=int32)
grid_x.eval():
 [[[[0]]

  [[1]]

  [[2]]

  [[3]]]


 [[[0]]

  [[1]]

  [[2]]

  [[3]]]


 [[[0]]

  [[1]]

  [[2]]

  [[3]]]]
grid: Tensor("Cast_29:0", shape=(3, 4, 1, 2), dtype=float32)
grid.eval(): [[[[0. 0.]]

  [[1. 0.]]

  [[2. 0.]]

  [[3. 0.]]]


 [[[0. 1.]]

  [[1. 1.]]

  [[2. 1.]]

  [[3. 1.]]]


 [[[0. 2.]]

  [[1. 2.]]

  [[2. 2.]]

  [[3. 2.]]]]
